In [ ]:
# [1] Setup Environment & Clone Repo
import os
import shutil

# Clean up if needed (for repeated runs)
if os.path.exists('/kaggle/working/ai_project'):
    shutil.rmtree('/kaggle/working/ai_project')

if os.path.exists('/kaggle/working'):
    os.chdir('/kaggle/working')

!git clone https://github.com/yangzeha/ai_project.git
%cd ai_project
!apt-get update && apt-get install -y libsparsehash-dev

In [ ]:
# [2] Compile MSBE (C++)
# This tool is needed for biclique mining
!g++ -O3 Similar-Biclique-Idx-main/main.cpp -o Similar-Biclique-Idx-main/msbe
!chmod +x Similar-Biclique-Idx-main/msbe

In [ ]:
# [3] Download & Process Yelp2018 Dataset
!python TSB_CL_Project/prepare_yelp2018.py

In [ ]:
# [4] Run Direct Fusion Comparison
# Compares TSB (Direct Fusion) vs LightGCN
!python TSB_CL_Project/quick_proof_direct.py

In [ ]:
# [5] Display Results
import matplotlib.pyplot as plt
import os
from IPython.display import Image, display

if os.path.exists('comparison_results.png'):
    display(Image(filename='comparison_results.png'))
else:
    print("No result image found. Check if the training finished successfully.")